<a href="https://colab.research.google.com/github/JayThibs/Machine-Learning-With-Tabular-Data/blob/master/Rossman_Store_Sales_Tabular_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rossmann Store Sales

## Description

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

## Problem Statement

Create a forecasting model to predict the number of sales in Rossmann stores.

In [0]:
%reload_ext autoreload
%autoreload 2

In [0]:
from fastai import *
from fastai.tabular import *
from fastai.basics import *
import os # for reading from the file

## Data Preparation / Feature Engineering

We will be using [external data](http://files.fast.ai/part2/lesson14/rossmann.tgz) along with the data provided in the Kaggle competition.

In [3]:
!wget -P /content/ http://files.fast.ai/part2/lesson14/rossmann.tgz

--2019-10-06 19:05:04--  http://files.fast.ai/part2/lesson14/rossmann.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7730448 (7.4M) [application/x-gtar-compressed]
Saving to: ‘/content/rossmann.tgz’

rossmann.tgz        100%[===================>]   7.37M  13.0MB/s    in 0.6s    

2019-10-06 19:05:05 (13.0 MB/s) - ‘/content/rossmann.tgz’ saved [7730448/7730448]

/bin/bash: untar: command not found


In [0]:
!tar -xf /content/rossmann.tgz

In [0]:
!rm -rf /content/rossmann.tgz

In [12]:
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend',
               'weather', 'test']

tables = [pd.read_csv(f'{fname}.csv', low_memory=False) \
          for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train), len(test) # number of train and test examples

(1017209, 41088)